In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.preprocessing import StandardScaler

In [5]:
df = pd.read_csv('Sample - Superstore.csv', encoding='latin1')

print(df.head())
print(df.info())


   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   Sout

# Step 2: Standardize Text Columns

In [8]:
text_cols = ['Ship Mode', 'Customer ID', 'Customer Name',
             'Segment', 'Country', 'City', 'State',
             'Region', 'Category', 'Sub-Category', 'Product Name']

for col in text_cols:
    df[col] = df[col].astype(str).str.lower().str.strip()
    df[col] = df[col].str.replace(f"[{string.punctuation}]", "", regex=True)

In [10]:
print(df[text_cols].head(10))

        Ship Mode Customer ID    Customer Name    Segment        Country  \
0    second class     cg12520      claire gute   consumer  united states   
1    second class     cg12520      claire gute   consumer  united states   
2    second class     dv13045  darrin van huff  corporate  united states   
3  standard class     so20335    sean odonnell   consumer  united states   
4  standard class     so20335    sean odonnell   consumer  united states   
5  standard class     bh11710  brosina hoffman   consumer  united states   
6  standard class     bh11710  brosina hoffman   consumer  united states   
7  standard class     bh11710  brosina hoffman   consumer  united states   
8  standard class     bh11710  brosina hoffman   consumer  united states   
9  standard class     bh11710  brosina hoffman   consumer  united states   

              City       State Region         Category Sub-Category  \
0        henderson    kentucky  south        furniture    bookcases   
1        henderson   

# Step 3: Check for Duplicates

In [13]:
# Are there any duplicates across all columns?
print(df.duplicated().sum())

# Drop them if found
df = df.drop_duplicates()

0


# Step 4: Outlier Check — Sales & Profit

In [16]:
# For Sales
Q1 = df['Sales'].quantile(0.25)
Q3 = df['Sales'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Sales Outlier bounds: {lower_bound} to {upper_bound}")

# Same for Profit
Q1_p = df['Profit'].quantile(0.25)
Q3_p = df['Profit'].quantile(0.75)
IQR_p = Q3_p - Q1_p

lower_bound_p = Q1_p - 1.5 * IQR_p
upper_bound_p = Q3_p + 1.5 * IQR_p

print(f"Profit Outlier bounds: {lower_bound_p} to {upper_bound_p}")


Sales Outlier bounds: -271.71000000000004 to 498.93
Profit Outlier bounds: -39.724125 to 70.816875


# Step 5: Optional — Feature Engineering

In [21]:
df['Profit Margin'] = np.where(df['Sales'] == 0, 0, df['Profit'] / df['Sales'])

In [23]:
print(df[['Sales', 'Profit', 'Profit Margin']].head(10))

      Sales    Profit  Profit Margin
0  261.9600   41.9136         0.1600
1  731.9400  219.5820         0.3000
2   14.6200    6.8714         0.4700
3  957.5775 -383.0310        -0.4000
4   22.3680    2.5164         0.1125
5   48.8600   14.1694         0.2900
6    7.2800    1.9656         0.2700
7  907.1520   90.7152         0.1000
8   18.5040    5.7825         0.3125
9  114.9000   34.4700         0.3000


# final check 

In [26]:
print(df[df['Sales'] < 0])
print(df[df['Quantity'] < 0])


Empty DataFrame
Columns: [Row ID, Order ID, Order Date, Ship Date, Ship Mode, Customer ID, Customer Name, Segment, Country, City, State, Postal Code, Region, Product ID, Category, Sub-Category, Product Name, Sales, Quantity, Discount, Profit, Profit Margin]
Index: []

[0 rows x 22 columns]
Empty DataFrame
Columns: [Row ID, Order ID, Order Date, Ship Date, Ship Mode, Customer ID, Customer Name, Segment, Country, City, State, Postal Code, Region, Product ID, Category, Sub-Category, Product Name, Sales, Quantity, Discount, Profit, Profit Margin]
Index: []

[0 rows x 22 columns]


In [30]:
# for discount column check 
print(df[df['Discount'] < 0])
print(df[df['Discount'] > 1])  

Empty DataFrame
Columns: [Row ID, Order ID, Order Date, Ship Date, Ship Mode, Customer ID, Customer Name, Segment, Country, City, State, Postal Code, Region, Product ID, Category, Sub-Category, Product Name, Sales, Quantity, Discount, Profit, Profit Margin]
Index: []

[0 rows x 22 columns]
Empty DataFrame
Columns: [Row ID, Order ID, Order Date, Ship Date, Ship Mode, Customer ID, Customer Name, Segment, Country, City, State, Postal Code, Region, Product ID, Category, Sub-Category, Product Name, Sales, Quantity, Discount, Profit, Profit Margin]
Index: []

[0 rows x 22 columns]


In [32]:
#Check for duplicates on business keys
print(df.duplicated(subset=['Order ID', 'Product ID']).sum())


8


In [34]:
# Find the duplicate pairs
dupes = df[df.duplicated(subset=['Order ID', 'Product ID'], keep=False)]

# Sort for easy reading
dupes = dupes.sort_values(by=['Order ID', 'Product ID'])

print(dupes)


      Row ID        Order ID  Order Date   Ship Date       Ship Mode  \
6498    6499  CA-2015-103135   7/24/2015   7/28/2015  standard class   
6500    6501  CA-2015-103135   7/24/2015   7/28/2015  standard class   
350      351  CA-2016-129714    9/1/2016    9/3/2016     first class   
352      353  CA-2016-129714    9/1/2016    9/3/2016     first class   
1300    1301  CA-2016-137043  12/23/2016  12/25/2016    second class   
1301    1302  CA-2016-137043  12/23/2016  12/25/2016    second class   
9168    9169  CA-2016-140571   3/15/2016   3/19/2016  standard class   
9169    9170  CA-2016-140571   3/15/2016   3/19/2016  standard class   
7881    7882  CA-2017-118017   12/3/2017   12/6/2017    second class   
7882    7883  CA-2017-118017   12/3/2017   12/6/2017    second class   
3183    3184  CA-2017-152912   11/9/2017  11/12/2017    second class   
3184    3185  CA-2017-152912   11/9/2017  11/12/2017    second class   
3405    3406  US-2014-150119   4/23/2014   4/27/2014  standard c

In [38]:
# Combine partial lines
df_cleaned = df.groupby(['Order ID', 'Product ID'], as_index=False).agg({
    'Order Date': 'first',
    'Ship Date': 'first',
    'Ship Mode': 'first',
    'Customer ID': 'first',
    'Customer Name': 'first',
    'Segment': 'first',
    'Country': 'first',
    'City': 'first',
    'State': 'first',
    'Postal Code': 'first',
    'Region': 'first',
    'Category': 'first',
    'Sub-Category': 'first',
    'Product Name': 'first',
    'Sales': 'sum',
    'Quantity': 'sum',
    'Discount': 'mean',
    'Profit': 'sum'
})

# Recalculate profit margin
df_cleaned['Profit Margin'] = np.where(
    df_cleaned['Sales'] == 0, 0, df_cleaned['Profit'] / df_cleaned['Sales']
)


In [40]:
print(df_cleaned.head())
print(df_cleaned[df_cleaned.duplicated(subset=['Order ID', 'Product ID'])])


         Order ID       Product ID Order Date  Ship Date       Ship Mode  \
0  CA-2014-100006  TEC-PH-10002075   9/7/2014  9/13/2014  standard class   
1  CA-2014-100090  FUR-TA-10003715   7/8/2014  7/12/2014  standard class   
2  CA-2014-100090  OFF-BI-10001597   7/8/2014  7/12/2014  standard class   
3  CA-2014-100293  OFF-PA-10000176  3/14/2014  3/18/2014  standard class   
4  CA-2014-100328  OFF-BI-10000343  1/28/2014   2/3/2014  standard class   

  Customer ID     Customer Name      Segment        Country           City  \
0     dk13375       dennis kane     consumer  united states  new york city   
1     eb13705        ed braxton    corporate  united states  san francisco   
2     eb13705        ed braxton    corporate  united states  san francisco   
3     nf18475  neil französisch  home office  united states   jacksonville   
4     jc15340   jasper cacioppo     consumer  united states  new york city   

   ... Postal Code  Region         Category Sub-Category  \
0  ...       1

In [42]:
#Check categorical consistency
print(df['Segment'].unique())
print(df['Category'].unique())
print(df['Sub-Category'].unique())
print(df['Ship Mode'].unique())

['consumer' 'corporate' 'home office']
['furniture' 'office supplies' 'technology']
['bookcases' 'chairs' 'labels' 'tables' 'storage' 'furnishings' 'art'
 'phones' 'binders' 'appliances' 'paper' 'accessories' 'envelopes'
 'fasteners' 'supplies' 'machines' 'copiers']
['second class' 'standard class' 'first class' 'same day']


In [50]:
df_cleaned.to_csv("superstore_cleaned.csv", index=False)


In [52]:
df_cleaned.to_csv("superstore_cleaned.csv", index=False, encoding='utf-8-sig')
